# 236605 Final Project

## KNN Demonstration 

In [1]:
import sklearn as sk
import matplotlib.pyplot as plt
import torchvision.transforms as tvtf
from ECG_multi_lead_dataloader import *
import transforms as tf
import torchvision
import torch
import knn_classifier as knn
import os

In [2]:
# Define the transforms that should be applied to each ECG record before returning it
tf_ds = tvtf.Compose([
    tf.ECG_tuple_transform(-1) # Reshape to 1D Tensor
])

In [5]:
root_dir = r'/home/noamkeidar/vadimDB/'

ECG_test = ECG_Multilead_Dataset(root_dir=root_dir, transform=tf_ds) # For KNN demo

In [6]:
# Define how much data to load (only use a subset for speed)
num_train = 35000
num_test = 1000
batch_size = 10000

# Training dataset & loader
ds_train = tf.SubsetDataset(ECG_test, num_train)  #(train=True, transform=tf_ds)
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=False)

# Test dataset & loader
ds_test = tf.SubsetDataset(ECG_test, num_test, offset=num_train)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size)

# Get all test data to predict in one go
test_iter = iter(dl_test)
x_test, y_test = test_iter.next()

In [7]:
knn_classifier = knn.KNNClassifier(k=10)
knn_classifier.train(dl_train)
y_pred = knn_classifier.predict(x_test)

# Calculate accuracy
accuracy = knn.accuracy(y_test, y_pred)
print(f'Accuracy: {accuracy*100:.2f}%')

Accuracy: 85.70%


## Digitized 12 Lead ECG classification

Clean up the environment from the previous exmple to free the memory (if necessary):

In [41]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  n


Nothing done.


In [42]:
import torch
torch.cuda.empty_cache()

### Imports and loadings

Imports and preperations, change root_dir here to the directory of the data pickles

In [43]:
import torch
import models
import transforms as tf
import matplotlib.pyplot as plt
from ECG_multi_lead_dataloader import *

##### Change root direrctory here #####
root_dir = r'/home/noamkeidar/vadimDB/'

%load_ext autoreload
%autoreload 2

plt.rcParams.update({'font.size': 12})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using device: cuda


Load the database (might take some minutes)

In [44]:
ds = ECG_Multilead_Dataset(root_dir=root_dir)

Prepare the dataloaders

In [45]:
# Define how much data to load (only use a subset for speed)

# for real training:
num_train = 35000
# for small set overfit experiment: 
#num_train = 3500
num_test = 1000
batch_size = 512

# Training dataset & loader
ds_train = tf.SubsetDataset(ds, num_train)  #(train=True, transform=tf_ds)
dl_train = torch.utils.data.DataLoader(ds_train,batch_size= batch_size,shuffle=True)

# Test dataset & loader
ds_test = tf.SubsetDataset(ds, num_test, offset= num_train)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size)

Let's see what did we load:

In [46]:
x, y = iter(dl_train).next()
x1, x2 = x

print('Long lead data of shape: ', x2.shape)
print('Short lead data of shape: ', x1.shape)
print('Labels of shape: ', y.shape)

Long lead data of shape:  torch.Size([512, 1, 5000])
Short lead data of shape:  torch.Size([512, 12, 1250])
Labels of shape:  torch.Size([512])


### Model creation

Use this box to determine the archetecture of the digitized ECG classifing model. Note that the input data is tuples structured $(x_1,x_2)$, on which $x_1$ is a 12 on $N$ matrix containing digitized signals of short 12 lead ECG and $x_2$ is a 1 on $4\cdot N$ matrix containing the long lead. Both inputs enter the model through 1d CNNs and combined to a single set of features before finally being cassified by a simple feedforward NN. 

In [53]:
### CNNs structure: 

# num of channels and kernel length in each layer of each branch, note that list lengths must correspond 
short_hidden_channels = [16, 32, 64, 128, 256, 512]
long_hidden_channels  = [4, 8, 16, 32, 64, 128, 256, 512]
short_kernel_lengths = [5]*6
long_kernel_lengths = [5]*8

# which tricks to use: dropout, stride, batch normalization and dilation 
short_dropout = 0.5
long_dropout = 0.5
short_stride = 2
long_stride = 2
short_dilation = 1
long_dilation = 1
short_batch_norm = True
long_batch_norm = True

# enter input length here
short_input_length = 1250
long_input_length = 5000

### FC net structure:

# num of hidden units in every FC layer
fc_hidden_dims = [128]

# num of output classess 
num_of_classes = 2

build the model:

In [54]:
model = models.Ecg12LeadNet(short_hidden_channels, long_hidden_channels,
                 short_kernel_lengths, long_kernel_lengths,
                 fc_hidden_dims,
                 short_dropout, long_dropout,
                 short_stride, long_stride,
                 short_dilation, long_dilation,
                 short_batch_norm, long_batch_norm,
                 short_input_length, long_input_length,
                 num_of_classes).to(device)

print(model)

Ecg12LeadNet(
  (short_cnn): ConvNet(
    (cnn): Sequential(
      (0): Conv1d(12, 16, kernel_size=(5,), stride=(2,))
      (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Dropout(p=0.5)
      (3): ReLU()
      (4): Conv1d(16, 32, kernel_size=(5,), stride=(2,))
      (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): Dropout(p=0.5)
      (7): ReLU()
      (8): Conv1d(32, 64, kernel_size=(5,), stride=(2,))
      (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): Dropout(p=0.5)
      (11): ReLU()
      (12): Conv1d(64, 128, kernel_size=(5,), stride=(2,))
      (13): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (14): Dropout(p=0.5)
      (15): ReLU()
      (16): Conv1d(128, 256, kernel_size=(5,), stride=(2,))
      (17): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (18)

Try the model on a single batch to make sure the dimentions fit:

In [55]:
x_try = (x1.to(device, dtype=torch.float), x2.to(device, dtype=torch.float))
y_pred = model(x_try)
print('Output batch size is:', y_pred.shape[0], ', and number of class scores:', y_pred.shape[1],'\n')

num_correct = torch.sum((y_pred > 0).flatten() == (y.to(device, dtype=torch.long)==1))
print(100*num_correct.item()/len(y), '% Accurecy... maybe we should consider training the model')

Output batch size is: 512 , and number of class scores: 1 

26.953125 % Accurecy... maybe we should consider training the model


### Let the game begin - Training

This training section is based on the course abstruct Trainer class from HW 3. We have implemented a custom class for our model inhereting from Trainer.

In [62]:
import torch.nn as nn
import torch.optim as optim
from training import Ecg12LeadNetTrainerBinary

torch.manual_seed(42)

lr = 0.001
num_epochs = 25

loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
trainer = Ecg12LeadNetTrainerBinary(model, loss_fn, optimizer, device)

fitResult = trainer.fit(dl_train, dl_test, num_epochs, checkpoints=r'checkpoints/Ecg12LeadNet',
                        early_stopping=10, print_every=1)

--- EPOCH 1/25 ---
test_batch (Avg. Loss 0.092, Accuracy 96.4): 100%|██████████| 2/2 [00:00<00:00,  7.82it/s]
--- EPOCH 2/25 ---
test_batch (Avg. Loss 0.095, Accuracy 96.3): 100%|██████████| 2/2 [00:00<00:00,  8.75it/s]
--- EPOCH 3/25 ---
test_batch (Avg. Loss 0.115, Accuracy 96.7): 100%|██████████| 2/2 [00:00<00:00,  7.92it/s]
*** Saved checkpoint checkpoints/Ecg12LeadNet.pt at epoch 3
--- EPOCH 4/25 ---
test_batch (Avg. Loss 0.112, Accuracy 95.6): 100%|██████████| 2/2 [00:00<00:00,  8.71it/s]
--- EPOCH 5/25 ---
test_batch (Avg. Loss 0.091, Accuracy 96.7): 100%|██████████| 2/2 [00:00<00:00,  7.76it/s]
--- EPOCH 6/25 ---
test_batch (Avg. Loss 0.094, Accuracy 95.8): 100%|██████████| 2/2 [00:00<00:00,  8.81it/s]
--- EPOCH 7/25 ---
test_batch (Avg. Loss 0.125, Accuracy 95.2): 100%|██████████| 2/2 [00:00<00:00,  8.74it/s]
--- EPOCH 8/25 ---
test_batch (Avg. Loss 0.107, Accuracy 95.7): 100%|██████████| 2/2 [00:00<00:00,  8.70it/s]
--- EPOCH 9/25 ---
test_batch (Avg. Loss 0.095, Accuracy 96.

In [64]:
test_result = trainer.test_epoch(dl_test, verbose=True)
print('Test accurecy is: ', test_result[1], '%')

test_batch (Avg. Loss 0.081, Accuracy 97.5): 100%|██████████| 2/2 [00:00<00:00,  6.21it/s]
Test accurecy is:  97.5 %
